In [ ]:
import sys
extra_pathes = (
    './src', 
    '../egrid/src', 
    '../graphparser/src')
for path in extra_pathes:
    if path not in sys.path:
        sys.path.append(path)
import dssex
from egrid import create_objects, make_data_frames, check_frames, model_from_frames
import pandas as pd
pd.set_option('display.max_colwidth', None)

## Data

In [ ]:
schema = """
                   (                                 ~~~
               +-->(                                 ~~~
slack +--------+   (-Branch-------------+ n +-------||||| heating_
        Tlink=tap     y_lo=1k-1kj         |                P10=200
                      y_tr=1µ+1µj         |
                                          |        \<-+->/
                                          |           |
                        _cap ||---------+ n +-------((~)) motor_
                          Q10=-10                          P10=160
                          Exp_v_q=2                        Q10=10

#.Deft(id=tap type=var min=-16 max=16 value=0 is_discrete=True cost=.03)
#.Defk(id=kcap type=var min=0 max=5 value=0 is_discrete=True cost=.05)
#.Klink(id_of_injection=cap id_of_factor=kcap part=q)
#.Vlimit(min=.95)
"""

## Data Frames

In [ ]:
frames = make_data_frames(create_objects(schema))

## Print Frames

In [ ]:
for key,df in frames.items():
    if not df.empty:
        print()
        print(key)
        display(df)

## Check Data

In [ ]:
pd.DataFrame.from_records(check_frames(frames), columns=['message','level'])

## Create Model

Function **model_from_frames** accepts a second optional argument which is a float value for maximum admittance of a branch. The default values is **1e5**. Branches having an admittance exceeding this value are treated like short circuits. Adding a branch to the model without giving a value for attribute **y_tr** creates such a branch which models a closed switch.

In [ ]:
model = model_from_frames(frames)

## Optimize

**step_params** is a list of dictionaries (dict), all entries are optional, processed key-value-pairs are:

- 'objectives': str, objectives
- 'constraints': str, constraints
- 'floss': float, factor for losses

If step_params is empty the function calculates power flow only.
Each dict triggers an optimization step.

Value for keys **objectives** is a string made of characters:

- 'P' for active power
- 'Q' for reactive power
- 'I' for electric current
- 'V' for voltage
- 'U' for voltage violation
- 'L' for losses of branches
- 'C' for cost
- 'T' of model.terms

Value for keys **constraint** is a string made of characters:

- 'P' keeping the initial values of active power at the location of given active power values during this step
- 'Q' keeping the initial values of reactive power at the location of given reactive power values during this step
- 'I' keeping the initial values of electric current at the location of given current values during this step
- 'V' keeping the initial values of voltages at the location of given voltage values during this step
- 'U' considering voltage limits

In [ ]:
step_params=[
    # first step: minimize voltage violations
    dict(objectives='U', constraints='U'),
    # second step: minimize cost
    dict(objectives='CL', constraints='U', floss=2)]
messages, results = dssex.estimate(model, step_params=step_params)

## Result

In [ ]:
if not messages.empty:
    print('\n\n')
    print('Messages')
    display(messages)
last_index = len(results) - 2
for idx, res in enumerate(results, -1):
    print('\n\n')
    print(f'{"RESULT >> " if idx==last_index else ""}Optimization Step: {idx}')
    for name in ['nodes', 'branches', 'injections']:
        print()
        print(name.title())
        display(res[name])

## Model
The model is made from a multiline text string. The string has a pseudo graphic part and a footer part.

For example:

```
slack ---------------Branch-------------- n ------------- heating
        Tlink=tap     y_lo=1k-1kj         |                P10=200
                      y_tr=1µ+1µj         |
                                          |
                         cap ------------ n ------------- motor
                          Q10=-10                          P10=160
                          Exp_v_q=2                        Q10=10

#.Deft(id=tap type=var min=-16 max=16 value=0 is_discrete=True cost=.03)
#.Defk(id=kcap type=var min=0 max=5 value=0 is_discrete=True cost=.05)
#.Klink(id_of_injection=cap id_of_factor=kcap part=q)
#.Vlimit(min=.95)
```

Pseudographic rules:

- entities in the multiline text are connectivity nodes (including slack nodes), branches, and injections
- IDs are word characters and numbers, underscores inside
- IDs starting wit "n" are connectivity nodes
- IDs starting wit "slack" are slack nodes
- branches are entities between two nodes
- injection IDs are adjacent to one connectivity node only
- text lines are 
    - entity lines, left most entity is a node or device ID, e.g. `n0`
    - attribute lines, left most detected pattern is an attribute e.g. `P=10` or `Vlimit.min=.98`
    - footer, starts with '#.', can also be a header or in the middle
    - comments, first character is '#' (hash), second is not '.' (dot)
    - blank lines, lines having no other type
- IDs are sequences of alphanumeric characters including underscore, leading/trailing underscore characters are not part of the ID
- IDs with leading or trailing underscore indicate that the entity is not connected to entities left/right, this is useful for placing injections in one entity line, the underscore is added to nodes or injection IDs
- attributes IDs consit of alphanumeric characters, underscore and characters necessary to insert numbers, attribute IDs are followed by an equal sign "="
- attribute values follow the equal sign, attribute values must be quoted if they contain space characters
- attribute lines are associated to text lines which they follow or preceed without blank lines
- blank lines separate entity lines (with their preceeding or following attribute lines)
- other characters including space have no meaning
- attributes are associated to entities whose ID share the same column like the first character of the attribute ID, IDs of attributes associated to terminals do not share their first column with any entity

The pseudographic displayed above creates the same model like this string:
```
slack Branch n
      y_lo=1k-1kj
      y_tr=1µ+1µj

heating n
P10=200

cap n
Q10=-10
Exp_v_q=2

motor n
P10=160
Q10=10

#.Tlink(id_of_node=slack id_of_branch=Branch id_of_factor=tap)
#.Deft(id=tap type=var min=-16 max=16 value=0 is_discrete=True cost=.03)
#.Defk(id=kcap type=var min=0 max=5 value=0 is_discrete=True cost=.05)
#.Klink(id_of_injection=cap id_of_factor=kcap part=q)
#.Vlimit(min=.95)
```

## Elements of Model
### Slacknode
Tag for a slack node.

Attributes:
- id_of_node: str, optional, default 'slack', identifier of the slack node
- V: complex, optional, default 1+0j, voltage at slack node

Examples:

- in footer 
```
#.Slacknode(id=slack01 V=.95+.2j)
```
- in graphic 
```
slack01
 V=.95+.2j
```
### Branch
Model of an electrical device having two terminals e.g. transformers, windings of multi-winding transformers, lines, closed switch.

Attributes:

- id: str, id of branch
- id_of_node_A: str, id of node at side A
- id_of_node_B: str, id of node at side B
- y_lo: complex, optional, default value is complex(numpy.inf, numpy.inf), longitudinal admittance
- y_tr: complex, optional, default value 0j, transversal admittance

Examples:

- in footer 
```
#.Branch(id=line id_of_node_A=n0 id_of_node_B=n1 y_lo=1e6+0.9e6j y_tr1e-6+0.9e-6j)
```
- in graphic 
```
n0    line                 n1
       y_lo=1e6+0.9e6j
       y_tr1e-6+0.9e-6j
```
### Injection
Model of an electrical one-terminal-device e.g. consumers (positiv and negative loads), PQ- and PV-generators, batteries and shunt capacitors.

Attributes:

- id: str, unique identifier of injection
- id_of_node: str, id of connected node
- P10: float, optional, default value 0, active power when magnitude of voltage is 1.0 pu, the value is the sum for all three phases
- Q10: float, optional, default value 0, reactive power when magnitude of voltage is 1.0 pu, the value is the sum for all three phases
- Exp_v_p: float, optional, default value 0, exponent for voltage dependency of active power, 0.0 active power P is independent from voltage-magnitude e.g. generators, 2.0 for constant conductance
- Exp_v_q: float, optional, default value 0, exponent for voltage dependency of active power, 0.0 reactive power is independent from voltage-magnitude e.g. generators, 2.0 for constant susceptance

Examples:

- in footer 
```
#.Injection(id=mill id_of_node=n0 P10=.5 Q10=.2 Exp_v_p=1 Exp_v_q=1)
```
- in graphic 
```
n0   mill
      P10=.5 Q10=.2
      Exp_v_p=1 Exp_v_q=1
```
### PValue
Value of (measured) active power.

Attributes:

- id_of_batch: str, unique identifier of the measurement group
- P: float, optional, default value is 0, active power, sum of all three phases
- direction: -1 or 1, optional, default value 1, 1 if power flows from node into device, -1 otherwise
- cost: float, optional, default value 0, cost

Examples:

- in footer 
```
#.PValue(id_of_batch=n0_line1 P=13 direction=1 cost=.3)
```
- in graphic (at terminal, between node and branch/injection)
```
n0             line1
    P=13
    P.cost=.3
```
### QValue
> Value of (measured) reactive power.

Attributes:

- id_of_batch: str, unique identifier of the measurement group
- Q: float, optional, default value is 0, reactive power, sum of all three phases
- direction: -1 or 1, optional, default value 1, 1 if power flows from node into device, -1 otherwise
- cost: float, optional, default value 0, cost

Examples:

- in footer 
```
#.QValue(id_of_batch=n0_line1 Q=7 direction=1 cost=.3)
```
- in graphic (at terminal, between node and branch/injection)
```
n0             line1
    Q=7
    Q.cost=.05
```
### IValue
(Measured) magnitude of electric current.

Attributes:

- id_of_batch: str, unique identifier of the measurement group
- I: float, optional, default value is 0, reactive power, sum of all three phases

Examples:

- in footer 
```
#.IValue(id_of_batch=n0_line1 I=8)
```
- in graphic (at terminal, between node and branch/injection)
```
n0             line1
    I=8
```
### Output
Measured terminal or terminal of a device which is part of a group of devices whose flow is measured.

Attributes:

- id_of_batch: str, unique identifier of the batch
- id_of_device: str, id referencing a branch or an injection
- id_of_node: str, optional, default value None, id of the connected node, 'None' if at injection

Examples:

- in footer 
```
#.Output(id_of_batch=n0_line1 id_of_device=line1, id_of_node=n0)
```
- automatically created by the builder if flow values are placed in graphic, __Output__ is not part of graphic data
### Vvalue
Measured magnitude or setpoint of electric voltage.

Attributes:

- id_of_node: str, unique identifier of node the voltage was measured at or the setpoint is for
- Vvalue: float, optional, default value 1.0, magnitude of electric voltage

Examples:

- in footer 
```
#.Vvalue(id_node=n0 V=.97)
```
- in graphic (at node)
```
n0
V=.97
```
### Vlimit, Defvl
Limits of node voltage.

Attributes:

- id_of_node: str, optional, defaults to "", identifier of connectivity node, empty string "" addresses all nodes
- min: float, optional, default 0.9, smallest possible magnitude of the voltage at node
- max: float, optional, default 1.1, greatest possible magnitude of the voltage at node
- step: int, optional, default -1, index of optimization step, -1 for all steps

Examples:

- in footer 
```
#.Vlimit(id_of_node=n0 min=.95 max=1.03 step=0)
```
- in footer for multiple nodes and/or multiple steps
```
#.Defvl(id_of_node(n0 n27) min=.95 max=1.03 step(0 1))
```
- in graphic (at node)
```
n0               line1
Vlimit.min=.95
Vlimit.max=1.03
```
### Factor, Defk, Deft
Decision variable or parameter. Factor introduces decision variables or parameters scaling active or reactive power of injections and variables for tap positions associated to terminals of branches or to injections.

Attributes:

- id: str, unique idendifier of factor
- type: 'var'|'const', optional, default value 'var', decision variable or parameter
- id_of_source: str, optional, default value '_default_', id of scaling factor (previous optimization step) for initialization
- value: float, optional, default vaLue 1, used by initialization if no source factor in previous optimization step
- min: float, optional, default value -inf, smallest possible value
- max: float, optional, default value inf, greatest possible value
- is_discrete: bool, optional, default is False, no values after decimal point if True, input for solver accepted by MINLP solvers
- m: float, optional, default 1., dy/dx, effective multiplier is a linear function f(x) = mx + n, m is the increase of that linear function, value is applied for tap factors only
- n: float, optiona, default 0., effective multiplier is a linear function f(x) = mx + n, n is f(0), value is applied for tap factors only
- step: int, optional, default value -1, index of optimization step, defined for each step if set to -1
- cost: float, optional, default 1., cost of change (for Volt-Var-Control)

Examples:

- in footer for scaling of injection
```
#.Factor(id=kp type=var value=.8 min=0 max=1 is_discrete=False step=-1 cost=1)
```
- in footer for taps
```
#.Factor(id=tap0 type=const value=3 min=-9 max=9 is_discrete=True m=.00625 n=1 step=-1 cost=1)
```
- in footer for multiple scaling factors and/or multiple steps (Defk shares defaults with Factor)
```
#.Defk(id(kp kq) type=var value=.8 min=0 max=1 is_discrete=False step(0 1) cost=1)
```
- in footer for multiple taps factors and/or multiple steps (defaults deviating from Factor: 'type' default is'const', 'value' default is 0, 'min' default is -16, 'max' default is 16, 'm' default is -0.00625, 'n' default is 1)
```
#.Deft(id(tap0 tap1) type=const value=0 min=-16 max=16 is_discrete=True m=-.00625 n=1 step(0 1) cost=0)
```
- Factor cannot be input in graphic
### Klink
Logical connection between injection and a factor.

Attributes:

- id_of_injection: str|tuple_of_str, identifier of injection
- part: 'p'|'q'|tuple_of_p_q, identifies the attribute of the injection to multipy with ('p'/'q'- injected active/reactive power)
- id_of_factor: str|tuple_of_str, identifier of scaling factor to connect to, one identifier for each given value of argument 'part'
- step: int|tuple_of_int, optional, default value -1, addresses the optimization step, first optimization step has index 0, defined for each step if -1


Examples:

- in footer for single relation injection-part-factor
```
#.Klink(id_of_injection=mill part=p id_of_factor=kp step=0)
```
- in footer for three injections with one factor
```
#.Klink(id_of_injection(inj0 inj1 inj2) part(p q) id_of_factor(kpq kpq) step=(0 1))
```
### Tlink
Logical connection between a terminal of a branch and a factor.

Attributes:

- id_of_node: str|tuple_of_str, ID of connectivity node
- id_of_branch: str|tuple_of_str, identifier of branch
- id_of_factor: str|tuple_of_str, identifier of taps (terminal) factor to connect, id_of_node, id_of_branch, id_of_factor shall have the same number of elements
- step: int|tuple_of_int, optional, default value -1, addresses the optimization step, first optimization step has index 0, defined for each step if -1

Examples:

- in footer for single relation node-branch-factor
```
#.TLink(id_of_node=n0 id_of_branch=transformer0 id_of_factor=tap0 step=0)
```
- in footer for two relations node-branch-factor and three steps
```
#.TLink(id_of_node(n0 n1) id_of_branch(transformer0 transformer1) id_of_factor(tap0 tap1) step(0 1 2))
```
- in graphic (at terminal, between node and branch/injection, just for one specific step or for all steps with -1 which is default and does not need to be written)
```
n0              transformer0
     Tlink=tap0
     Tlink.step=0
```
### Defoterm
Define objective function term.

Attributes:

- fn: str, optional, default 'diff', name of function (currently the only option)
    - diff: objective function term returns a penalty. The penalty increases with the difference between the factors. If just one factor is given the term returns the difference to 0 as a penalty. 
- args: str | tuple_of_str, optional, default '', id of factor
- weight: float, optional, default 1.0, multiplier for term in objective function
- step: int | tuple_of_int, optional, default -1, optimization step, -1 means all steps"""

Examples:

- in footer for single relation node-branch-factor
```
#.Defoterm(fn=diff args(kp kq) weight=1 step=0)
```
- Defoterm cannot be input in graphic
